# Data Proprecessing + Analysis
---
텍스트 데이터들을 전처리 하고 필요한 분석들을 함께 진행합니다.

- 배틀넷
   - 시간 데이터 전처리
   - 공지 글 삭제
   - 중복 데이터 전처리
   - 헤비유저 파악
   
  
- 인벤
   - 시간 데이터 전처리
   - 공지 글 삭제
   - contents컬럼의 NaN 제거
   - 헤비유저 파악
   
   
- 배틀넷+인벤 불용어 전처리

## 1. 라이브러리 로딩

In [1]:
## 라이브러리 로딩

import pandas as pd # 판다스 
import numpy as np # 넘파이
import glob # 파일 여러개 불러올 때
import time, datetime
from datetime import datetime

import re # 정규표현식
import csv # 파일로 저장하기 위한 툴

import ast # Abstrac Syntex Tree - 스트링 처리된 리스트를 리스트로 다시 리턴할때

In [68]:
from tqdm import tqdm, trange
from tqdm import tqdm_notebook
from time import sleep

## 2. 데이터 올리기

### - 인벤 데이터 올리기

In [2]:
# Inven Data 

df1 = pd.read_csv("data/invenData_1~10000.csv", encoding = 'utf8', header=None, engine='python')
df2 = pd.read_csv("data/invenData_10001~50000.csv", encoding='utf8', header=None, engine='python')
df3 = pd.read_csv("data/invenData_50001~70000.csv", encoding='utf8', header=None, engine='python')
df4 = pd.read_csv("data/invenData_70001~85834.csv", encoding='utf8', header=None, engine='python')
df5 = pd.read_csv("data/invenData_85835~100000.csv", encoding='utf8', header=None, engine='python')
df6 = pd.read_csv("data/invenData_100001~120000.csv", encoding='utf8', header=None, engine='python')
df7 = pd.read_csv("data/invenData_120001~140000.csv", encoding='utf8', header=None, engine='python')
df8 = pd.read_csv("data/invenData_140001~152823.csv", encoding='utf8', header=None, engine='python')
df9 = pd.read_csv("data/invenData_152824~160100.csv", encoding='utf8', header=None, engine='python')

In [3]:
# 인벤 데이터 한꺼번에 합침
df_inven = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8, df9])

In [4]:
# 컬럼명 붙이기
df_inven.columns = ['link_id', 'timestamp', 'title', 'contents']

In [6]:
# NaN을 아예 길이없는 스트링으로 전환
df_inven['contents'].fillna("", inplace=True)

In [7]:
## 내용이 너무 짧다. 제목과 내용을 합친다.

# 시작 컬럼 포지션을 잡기
source_col_loc = df_inven.columns.get_loc('title') 

# 컬럼 기준으로 제목 + 내용 합치기
df_inven['contents_all'] = df_inven.iloc[:,source_col_loc:source_col_loc+2].apply(lambda x: ",".join(x.astype(str)), axis=1)

In [8]:
# 인덱스를 리셋시켜서 14만번대가 나오게 하도록 한다.
df_inven.reset_index(inplace=True)

### - 배틀넷 데이터 올리기

In [9]:
# 파일 읽기
df_battle = pd.read_csv("data/battlenetData.csv", encoding='utf8', header=None, engine='python')

In [10]:
df_battle.columns = ['link_id', 'user_id', 'timestamp', 'title', 'contents']

## 3. 데이터 전처리 - tidydata 만들기

### - 배틀넷 
---
### 시간 데이터 전처리

In [12]:
## index 1017번째 timestamp가 비어있었음. 
df_battle = df_battle[df_battle['timestamp'].notnull()]

In [13]:
# 글을 드랍하였기 떄문에 인덱스 리셋

df_battle.reset_index(inplace=True)

In [14]:
# 새로운 데이트타임 형식 만들기 - 스트링으로 들어와서 새로 만들어야 합니다.

new_dates = []

for i in range(5699):
    
    years = int(df_battle['timestamp'][i][0:4])
    months = int(df_battle['timestamp'][i][5:7])
    days = int(df_battle['timestamp'][i][8:10])
    hours = int(df_battle['timestamp'][i][11:13]) 
    minutes = int(df_battle['timestamp'][i][15:17])
    
    dates = datetime(years, months, days, hours, minutes)
    
    new_dates.append(dates)

In [15]:
df_battle["new_datetime"] = new_dates

In [16]:
df_battle.drop('timestamp',axis=1, inplace=True)

In [17]:
df_battle.head()

,index,link_id,user_id,title,contents,new_datetime
0,0,4564557252,비전력이부조카당,"아바투르 유저인데, 하나 건의합니다.","[아바투르 유저인데, 하나 건의합니다.새로 생긴 하나무라 맵...뭐 이것도 맵 자체...",2017-05-06 05:21:00
1,1,4567016696,네이팜데쓰,"매칭 시스템, 아무리 생각해도 문제가 많습니다",[매칭좀 제대로 해주세요.9연패가 말이 되는 처사입니까 며칠전에 16연승했다고 벌을...,2018-09-08 16:53:00
2,2,4567016695,souse,ㅈ같은 매칭좀 어캐좀 해봐요,[왜 내가 잘해도 ㅈ같이못하는 새키들때문에 게임을 져야하죠? 왜 심지어 못하는새키랑...,2018-09-08 16:22:00
3,3,4567026707,Best,아나 스킨관련..,[신 스킨 살무사 아나 수면총 색 고정말고. 스킨 색 대로 해주셧으면 좋겠습니다....,2018-09-08 15:13:00
4,4,4567036673,세월,5인팟 + 승률50%집착매칭 + 밸런스 파괴 = 빠대무간지옥,[승률 50% 만든다고 조합 !@내놓고5인팟은 여전히 솔팟 사이에서 개판치고다니고너...,2018-09-07 13:58:00


### - 그 외 중복 데이터 처리하기

In [18]:
# 블리자드 사에서 올린 공지는 제외

df_battle = df_battle[df_battle.user_id != "Blizzard Entertainment"]

In [19]:
# 글 완전 중복은 일단 제거합니다

df_battle['contents'].drop_duplicates(keep='first', inplace = True)

In [20]:
# 헤비유저 파악 혹은 글 연달아 올린 사람 파악하기

user_list = df_battle['user_id'].unique()

for uid in user_list[:15]:
    dfuid = df_battle[df_battle['user_id']==uid]    

# 전처리 되도록이면 판다스로 하기. 유저 아이디 중복을 찾은다음에. 시간 차이가 몇 안나는 것들은 빼기(몇 분 이상 차이나면 없애는 방향으로)

In [21]:
# 헤비 유저(빅마우스)들 (상위 10명)
print(user_list[0:10])

['비전력이부조카당' '네이팜데쓰' 'souse' 'Best' '세월' 'Ditto' 'JELAWAT' '요우몽' '파워디지몬'
 '인생이지겨워']


In [23]:
## 글 많이 쓴사람 : 요XX
len(df_battle[df_battle['user_id']=='요우몽'])

177

In [27]:
## 위에서 글을 많이 쓴 사람들 중에서 시간차가 거의 나지 않고 + 거의 같은 내용의 글 인덱스를 찾습니다.

print(df_battle[df_battle['link_id']==4566866705])
print(df_battle[df_battle['link_id']==4566857028])

     index     link_id user_id              title  \
109    109  4566866705     요우몽  화메 궁쿨 책정부터 잘못되어있음   

                                              contents        new_datetime  
109  [성스러운징벌이 100초 붉은가호가 1분인데 @%*!!# 성능 따져봐도 당연 붉은가... 2018-08-30 04:10:00  
     index     link_id user_id         title  \
189    189  4566857028   파워디지몬  HOTS 영웅리그 건의   

                                              contents        new_datetime  
189  [HOTS는 이전부터 영웅리그에 대한 문제가 많았습니다.크게 정리하자면 3가지가 있... 2018-08-21 06:01:00  


In [28]:
# 위에서 나온 결과를 기준으로 판단하여 중복 행 몇개를 삭제한다. - 뒷 인덱스를 삭제합니다. 새로 쓴 글이 의사를 더 잘 반영한다고 기준을 둠

df_battle.drop([189],inplace=True)
df_battle.drop([109],inplace=True)

In [29]:
df_battle.reset_index(inplace=True)

In [31]:
## 내용이 너무 짧다. 제목과 내용을 합칩니다.

# 시작 컬럼 포지션을 잡기
source_col_loc = df_battle.columns.get_loc('title') 

# 컬럼 기준으로 제목 + 내용 합치기
df_battle['contents_all'] = df_battle.iloc[:,source_col_loc:source_col_loc+2].apply(lambda x: ",".join(x.astype(str)), axis=1)

In [32]:
df_battle.head()

,level_0,index,link_id,user_id,title,contents,new_datetime,contents_all
0,0,0,4564557252,비전력이부조카당,"아바투르 유저인데, 하나 건의합니다.","[아바투르 유저인데, 하나 건의합니다.새로 생긴 하나무라 맵...뭐 이것도 맵 자체...",2017-05-06 05:21:00,"아바투르 유저인데, 하나 건의합니다.,[아바투르 유저인데, 하나 건의합니다.새로 생..."
1,1,1,4567016696,네이팜데쓰,"매칭 시스템, 아무리 생각해도 문제가 많습니다",[매칭좀 제대로 해주세요.9연패가 말이 되는 처사입니까 며칠전에 16연승했다고 벌을...,2018-09-08 16:53:00,"매칭 시스템, 아무리 생각해도 문제가 많습니다,[매칭좀 제대로 해주세요.9연패가 말..."
2,2,2,4567016695,souse,ㅈ같은 매칭좀 어캐좀 해봐요,[왜 내가 잘해도 ㅈ같이못하는 새키들때문에 게임을 져야하죠? 왜 심지어 못하는새키랑...,2018-09-08 16:22:00,"ㅈ같은 매칭좀 어캐좀 해봐요,[왜 내가 잘해도 ㅈ같이못하는 새키들때문에 게임을 져야..."
3,3,3,4567026707,Best,아나 스킨관련..,[신 스킨 살무사 아나 수면총 색 고정말고. 스킨 색 대로 해주셧으면 좋겠습니다....,2018-09-08 15:13:00,"아나 스킨관련..,[신 스킨 살무사 아나 수면총 색 고정말고. 스킨 색 대로 해..."
4,4,4,4567036673,세월,5인팟 + 승률50%집착매칭 + 밸런스 파괴 = 빠대무간지옥,[승률 50% 만든다고 조합 !@내놓고5인팟은 여전히 솔팟 사이에서 개판치고다니고너...,2018-09-07 13:58:00,"5인팟 + 승률50%집착매칭 + 밸런스 파괴 = 빠대무간지옥,[승률 50% 만든다고..."


In [33]:
# 가공이 끝나는 열들은 다 드랍합니다. (유저 아이디는 보안문제로 더이상 사용하지 않습니다.)

df_battle.drop('user_id',axis=1, inplace=True)
df_battle.drop('title',axis=1, inplace=True)
df_battle.drop('contents',axis=1, inplace=True)
df_battle.drop('level_0',axis=1, inplace=True)
df_battle.drop('index',axis=1, inplace=True)

In [34]:
# 년도와 월을 따로 만듭니다.
df_battle['Year'] = df_battle['new_datetime'].dt.year
df_battle['Month'] = df_battle['new_datetime'].dt.month

In [35]:
df_battle.head()

,link_id,new_datetime,contents_all,Year,Month
0,4564557252,2017-05-06 05:21:00,"아바투르 유저인데, 하나 건의합니다.,[아바투르 유저인데, 하나 건의합니다.새로 생...",2017,5
1,4567016696,2018-09-08 16:53:00,"매칭 시스템, 아무리 생각해도 문제가 많습니다,[매칭좀 제대로 해주세요.9연패가 말...",2018,9
2,4567016695,2018-09-08 16:22:00,"ㅈ같은 매칭좀 어캐좀 해봐요,[왜 내가 잘해도 ㅈ같이못하는 새키들때문에 게임을 져야...",2018,9
3,4567026707,2018-09-08 15:13:00,"아나 스킨관련..,[신 스킨 살무사 아나 수면총 색 고정말고. 스킨 색 대로 해...",2018,9
4,4567036673,2018-09-07 13:58:00,"5인팟 + 승률50%집착매칭 + 밸런스 파괴 = 빠대무간지옥,[승률 50% 만든다고...",2018,9


### - 인벤
---
### 시간 데이터 전처리

In [36]:
df_inven.head()

,index,link_id,timestamp,title,contents,contents_all
0,0,1214270,2018-08-30 16:06,댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다 보면 때로는 나와 조금 다른 의견...,"댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다..."
1,1,1214678,2018-09-05 11:56,9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 스톰 인벤 커뮤니티팀입니다.히어로즈 인벤에서는 매주 ...,"9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 ..."
2,2,1214899,2018-09-06 22:38,"[오시공] 김정민,문규리,침착맨,네클릿,옥냥이 메피스토 특별방송!",,"[오시공] 김정민,문규리,침착맨,네클릿,옥냥이 메피스토 특별방송!,"
3,3,1215180,2018-09-10 23:57,ㅋㅋㅋㅋ 장구벌레,,"ㅋㅋㅋㅋ 장구벌레,"
4,4,1215179,2018-09-10 23:52,캘타스 뭐가 더 비슷한가요?,타겜이지만 ㅎ.. 고민,"캘타스 뭐가 더 비슷한가요?,타겜이지만 ㅎ.. 고민"


In [37]:
# 새로운 데이트타임 형식 만들기

inven_new_dates = []

for i in range(len(df_inven)):
 
    inven_dates = datetime.strptime(df_inven.iloc[i,2], "%Y-%m-%d %H:%M")
    inven_new_dates.append(inven_dates)

In [38]:
# 시간 열을 추가하고 없앱니다.
df_inven["new_datetime"] = inven_new_dates
df_inven.drop('timestamp',axis=1, inplace=True)

In [42]:
# 분석에 필요없어진 열들을 다시 드랍합니다.

#df_inven.drop('title',axis=1, inplace=True)
#df_inven.drop('contents',axis=1, inplace=True)
df_inven.drop('index',axis=1, inplace=True)

In [43]:
df_inven.tail()

,link_id,contents_all,new_datetime
147578,1030725,"와 미친 오늘 진짜 치열하다 ㅋㅋㅋ,5,6 위 매치 진짜 미친듯이 치열하네의외의 꿀...",2017-09-02 19:19:00
147579,1030724,"뭐 안늙어죽으면 불멸자는 맞지만,불멸자들중에서도 조건부로맞아죽어도 살아날수있으니 개...",2017-09-02 19:18:00
147580,1030723,"히오스 n주년 이벤 해서,플레이어들이스킨 부문영웅 부문 창작해서 각 서버별로 상금걸...",2017-09-02 19:16:00
147581,1030722,"Cc에 강한 탱커 생겻으면,Cc 맞으면 체력회복",2017-09-02 19:13:00
147582,1030721,"이겜 영리하다보면 어느구간부터가 잘하는지 의문임,체감상 그마도 몇십유저 빼고 다 고...",2017-09-02 19:13:00


### 인벤 - 그 외 중복 데이터 처리하기

In [44]:
print(df_inven[df_inven["link_id"] == 1214678],"\n","="*80,
      df_inven[df_inven["link_id"] == 1214270],"\n","="*80,
      df_inven[df_inven["link_id"] == 1213632],"\n","="*80,
      df_inven[df_inven["link_id"] == 1214199],"\n","="*80,
      df_inven[df_inven["link_id"] == 1213035],"\n","="*80,
      df_inven[df_inven["link_id"] == 1212385],"\n","="*80,
      df_inven[df_inven["link_id"] == 1211671],"\n","="*80,
      df_inven[df_inven["link_id"] == 1210740],"\n","="*80,
      df_inven[df_inven["link_id"] == 1209936],"\n","="*80,
      df_inven[df_inven["link_id"] == 1208994],"\n","="*80,
      df_inven[df_inven["link_id"] == 1208372],"\n","="*80,
      df_inven[df_inven["link_id"] == 1207739],"\n","="*80,
      df_inven[df_inven["link_id"] == 1207110],"\n","="*80)

     link_id                                       contents_all  \
1    1214678  9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 ...   
460  1214678  9월 1주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 ...   

           new_datetime  
1   2018-09-05 11:56:00  
460 2018-09-05 11:56:00   
 ================================================================================      link_id                                       contents_all  \
0    1214270  댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다...   
821  1214270  댓글 차단 BETA가 적용됩니다.,안녕하세요. 인벤 커뮤니티팀입니다.인벤을 이용하다...   

           new_datetime  
0   2018-08-30 16:06:00  
821 2018-08-30 16:06:00   
 ================================================================================       link_id                                       contents_all  \
1390  1213632  8월 3주차 히어로즈 인벤 주간 이벤트 당첨자 발표,안녕하세요. 히어로즈 오브 더 ...   

            new_datetime  
1390 2018-08-22 16:38:00   
 ================================================================================ 

1, 460 / 0 ,821 / 1390, 881, 1921, 2510, 3140, 3990, 4695, 5531, 6109, 6672, 7226  글들은 커뮤니티 팀에서 올린 것으로 삭제한다.

(운영자 아이디로 검색하여 글번호 확인하였음)

In [46]:
# 운영팀에서 올린 공지는 제외
# 아이디를 직접 긁어오지 않았으므로, 웹상에서 직접 링크 아이디를 찾아 제거합니다.

df_inven.drop([0],inplace=True)
df_inven.drop([1],inplace=True)
df_inven.drop([460],inplace=True)
df_inven.drop([821],inplace=True)
df_inven.drop([881],inplace=True)
df_inven.drop([1390],inplace=True)
df_inven.drop([1921],inplace=True)
df_inven.drop([2510],inplace=True)
df_inven.drop([3140],inplace=True)
df_inven.drop([3990],inplace=True)
df_inven.drop([4695],inplace=True)
df_inven.drop([5531],inplace=True)
df_inven.drop([6109],inplace=True)
df_inven.drop([6672],inplace=True)
df_inven.drop([7226],inplace=True)

In [47]:
df_inven.reset_index(inplace=True)

In [49]:
df_inven.drop('index',axis=1, inplace=True)

In [50]:
# inven 컬럼 다시조정

df_inven = df_inven[['link_id','new_datetime','contents_all']]

In [64]:
# 년도와 월을 따로 만듭니다.

df_inven['Year'] = df_inven['new_datetime'].dt.year
df_inven['Month'] = df_inven['new_datetime'].dt.month

In [65]:
df_inven.head()

,link_id,new_datetime,contents_all,Year,Month
0,1214899,2018-09-06 22:38:00,오시공 김정민 문규리 침착맨 네클릿 옥냥이 메피스토 특별방송,2018,9
1,1215180,2018-09-10 23:57:00,ㅋㅋㅋㅋ 장구벌레,2018,9
2,1215179,2018-09-10 23:52:00,캘타스 뭐가 더 비슷한가요 타겜이지만 ㅎ 고민,2018,9
3,1215178,2018-09-10 23:52:00,메피 방금사서 해봤는데 너무사기임 궁각만 잘재고 쓰면 못해도 더블킬은 따고들...,2018,9
4,1215177,2018-09-10 23:50:00,제발 메피는 궁이랑 보호막특 너프 좀 두개가 안되면 궁이라도 궁자체도 최상...,2018,9


## 3. 데이터 전처리 - 특수문자 전처리

In [55]:
# 정규 표현식
p_http = re.compile("(((http(s?))\:\/\/)?)([0-9a-zA-Z\-]+\.)+[a-zA-Z]{2,6}(\:[0-9]+)?(\S*)")
p_char = re.compile("[^가-힣ㄱ-ㅎㅏ-ㅣa-zA-Z0-9ㅣ ]")
p_tag = re.compile("<.+>")
p_brac = re.compile("[[\]]")

In [56]:
# 정규 표현식 함수화

def patternRemover(doc):
    doc = p_http.sub(' ', doc)
    doc = p_char.sub(' ', doc)
    doc = p_tag.sub(' ', doc)
    doc = p_brac.sub(' ', doc)
    return doc

In [58]:
# 배틀넷 - 특수문자 전처리

for i in range(len(df_battle["contents_all"])):
    df_battle.iloc[i,2] = patternRemover(df_battle.iloc[i,2])

In [66]:
len(df_inven)

147568

In [70]:
# 인벤 - 특수문자 전처리

%%time

total = tqdm_notebook(range(147568))

for i in total:
    df_inven.iloc[i,2] = patternRemover(df_inven.iloc[i,2])

CPU times: user 36min 53s, sys: 1min 28s, total: 38min 22s
Wall time: 39min 26s


In [73]:
df_inven.iloc[140000,2]

## 초성 나왔음 초성도 제거해야해..

'그래서 이스프레이는 언제나오죠 나오면 바로삼 ㄹㅇ'

### 데이터 합치기

In [75]:
df_all = pd.concat([df_battle, df_inven])

In [77]:
df_all.reset_index(inplace=True)

In [81]:
df_all.drop('index',axis=1, inplace=True)

In [163]:
df_all.to_csv("predata.csv", encoding='utf-8')

In [160]:
df_all.drop('tokens_contents', axis=1, inplace=True)

In [162]:
df_all.tail()

,link_id,new_datetime,contents_all,Year,Month
153203,1030725,2017-09-02 19:19:00,와 미친 오늘 진짜 치열하다 ㅋㅋㅋ 5 6 위 매치 진짜 미친듯이 치열하네의외의 꿀...,2017,9
153204,1030724,2017-09-02 19:18:00,뭐 안늙어죽으면 불멸자는 맞지만 불멸자들중에서도 조건부로맞아죽어도 살아날수있으니 개...,2017,9
153205,1030723,2017-09-02 19:16:00,히오스 n주년 이벤 해서 플레이어들이스킨 부문영웅 부문 창작해서 각 서버별로 상금걸...,2017,9
153206,1030722,2017-09-02 19:13:00,Cc에 강한 탱커 생겻으면 Cc 맞으면 체력회복,2017,9
153207,1030721,2017-09-02 19:13:00,이겜 영리하다보면 어느구간부터가 잘하는지 의문임 체감상 그마도 몇십유저 빼고 다 고...,2017,9
